In [1]:
import numpy as np
import pandas as pd
from pydataset import data
from sklearn import svm
from sklearn import model_selection
from statsmodels.tools.eval_measures import mse
import seaborn as sns
from numpy import absolute
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from statsmodels.stats.diagnostic import normal_ad
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.stattools import durbin_watson
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score , mean_absolute_error
from sklearn import linear_model

In [2]:
lr = linear_model.LinearRegression()

In [3]:
df=pd.read_csv("CO2_a_clean.csv")

In [4]:
df.head()

,Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),CO2 Emissions(g/km)
0,ACURA,ILX,COMPACT,2.0,4.0,AS5,Z,9.9,6.7,8.5,33.0,196.0
1,ACURA,ILX,COMPACT,2.4,4.0,M6,Z,11.2,7.7,9.6,29.0,221.0
2,ACURA,ILX HYBRID,COMPACT,1.5,4.0,AV7,Z,6.0,5.8,5.9,48.0,136.0
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6.0,AS6,Z,12.7,9.1,11.1,25.0,255.0
4,ACURA,RDX AWD,SUV - SMALL,3.5,6.0,AS6,Z,12.1,8.7,10.6,27.0,244.0


In [5]:
# from sklearn.preprocessing import MinMaxScaler

# Create an instance of the MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 70))

# Assuming 'data' is your dataset and 'column' is the column you want to scale
# You need to reshape the column to a 2D array before scaling
column_to_scale = df['CO2 Emissions(g/km)'].values.reshape(-1, 1)

# Fit and transform the column using the scaler
scaled_column = scaler.fit_transform(column_to_scale)

# Replace the original column in the dataset with the scaled values
df['CO2 Emissions(g/km)'] = scaled_column

In [6]:
x = df.drop(['CO2 Emissions(g/km)', 'Model' , 'Vehicle Class'], axis=1)

In [7]:
# List of categorical column names
categorical_columns = ['Make','Transmission', 'Fuel Type']

# Use Pandas get_dummies to one-hot encode the categorical columns
x = pd.get_dummies(x, columns=categorical_columns)

# Now, all the specified categorical columns are one-hot encoded
x

,Engine Size(L),Cylinders,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),Make_ACURA,Make_ALFA ROMEO,Make_ASTON MARTIN,Make_AUDI,...,Transmission_AV7,Transmission_AV8,Transmission_M5,Transmission_M6,Transmission_M7,Fuel Type_D,Fuel Type_E,Fuel Type_N,Fuel Type_X,Fuel Type_Z
0,2.0,4.0,9.9,6.7,8.5,33.0,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,2.4,4.0,11.2,7.7,9.6,29.0,True,False,False,False,...,False,False,False,True,False,False,False,False,False,True
2,1.5,4.0,6.0,5.8,5.9,48.0,True,False,False,False,...,True,False,False,False,False,False,False,False,False,True
3,3.5,6.0,12.7,9.1,11.1,25.0,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,3.5,6.0,12.1,8.7,10.6,27.0,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7402,2.0,4.0,10.7,7.7,9.4,30.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
7403,2.0,4.0,11.2,8.3,9.9,29.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
7404,2.0,4.0,11.7,8.6,10.3,27.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
7405,2.0,4.0,11.2,8.3,9.9,29.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [8]:
y = df[['CO2 Emissions(g/km)']]
y

,CO2 Emissions(g/km)
0,16.431925
1,20.539906
2,6.572770
3,26.126761
4,24.319249
...,...
7402,20.211268
7403,22.347418
7404,23.661972
7405,22.347418


In [9]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42, shuffle=True)

In [10]:
X_train

,Engine Size(L),Cylinders,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),Make_ACURA,Make_ALFA ROMEO,Make_ASTON MARTIN,Make_AUDI,...,Transmission_AV7,Transmission_AV8,Transmission_M5,Transmission_M6,Transmission_M7,Fuel Type_D,Fuel Type_E,Fuel Type_N,Fuel Type_X,Fuel Type_Z
5420,4.0,8.0,14.1,8.9,11.8,24.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1223,2.0,4.0,10.8,7.9,9.5,30.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
210,2.0,4.0,9.9,7.6,8.9,32.0,False,False,False,True,...,False,True,False,False,False,False,False,False,False,True
3931,2.0,4.0,10.7,8.8,9.8,29.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
6312,2.5,4.0,12.3,9.4,11.0,26.0,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5191,3.5,6.0,11.1,8.1,9.8,29.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
5226,3.0,6.0,12.0,8.5,10.4,27.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
5390,3.0,6.0,12.4,8.9,10.8,26.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
860,4.4,8.0,16.2,10.9,13.8,20.0,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True


In [11]:
X_train.columns

Index(['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)',
       'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)',
       'Fuel Consumption Comb (mpg)', 'Make_ACURA', 'Make_ALFA ROMEO',
       'Make_ASTON MARTIN', 'Make_AUDI', 'Make_BENTLEY', 'Make_BMW',
       'Make_BUGATTI', 'Make_BUICK', 'Make_CADILLAC', 'Make_CHEVROLET',
       'Make_CHRYSLER', 'Make_DODGE', 'Make_FIAT', 'Make_FORD', 'Make_GENESIS',
       'Make_GMC', 'Make_HONDA', 'Make_HYUNDAI', 'Make_INFINITI',
       'Make_JAGUAR', 'Make_JEEP', 'Make_KIA', 'Make_LAMBORGHINI',
       'Make_LAND ROVER', 'Make_LEXUS', 'Make_LINCOLN', 'Make_MASERATI',
       'Make_MAZDA', 'Make_MERCEDES-BENZ', 'Make_MINI', 'Make_MITSUBISHI',
       'Make_NISSAN', 'Make_PORSCHE', 'Make_RAM', 'Make_ROLLS-ROYCE',
       'Make_SCION', 'Make_SMART', 'Make_SRT', 'Make_SUBARU', 'Make_TOYOTA',
       'Make_VOLKSWAGEN', 'Make_VOLVO', 'Transmission_A10', 'Transmission_A4',
       'Transmission_A5', 'Transmission_A6', 'Tra

In [12]:
df.head()

,Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),CO2 Emissions(g/km)
0,ACURA,ILX,COMPACT,2.0,4.0,AS5,Z,9.9,6.7,8.5,33.0,16.431925
1,ACURA,ILX,COMPACT,2.4,4.0,M6,Z,11.2,7.7,9.6,29.0,20.539906
2,ACURA,ILX HYBRID,COMPACT,1.5,4.0,AV7,Z,6.0,5.8,5.9,48.0,6.572770
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6.0,AS6,Z,12.7,9.1,11.1,25.0,26.126761
4,ACURA,RDX AWD,SUV - SMALL,3.5,6.0,AS6,Z,12.1,8.7,10.6,27.0,24.319249


In [13]:
y_train

,CO2 Emissions(g/km)
5420,29.413146
1223,20.704225
210,17.910798
3931,21.525822
6312,26.455399
...,...
5191,21.197183
5226,24.319249
5390,26.126761
860,37.464789


In [14]:
y_test_actual = y_test

In [15]:
regressor_lr = linear_model.LinearRegression()
regressor_lr.fit(X_train, y_train)

LinearRegression()

In [16]:
y_pred = regressor_lr.predict(X_test)

In [17]:
regressor_lr.score(X_train, y_train)

0.9906005304638739

In [18]:
regressor_lr.score(X_test, y_test)

0.991630209701751

In [19]:
mse = mean_squared_error(y_test, y_pred)

In [20]:
mse

0.7410773223155894

In [23]:
def model_evaluation(model, X_test, y_test, model_name):
    # Use the 'model' to make predictions on the test dataset 'X_test'.
    y_pred = model.predict(X_test)
    
    # Calculate various regression metrics:
    # 1. Mean Absolute Error (MAE)
    MAE =  mean_absolute_error(y_test, y_pred)
    # 2. Mean Squared Error (MSE)
    MSE = mean_squared_error(y_test, y_pred)
    # 3. Root Mean Squared Error (RMSE)
    RMSE = np.sqrt(MSE)
    # 4. R-squared Score (R2-Score)
    R2_Score =r2_score(y_test, y_pred)
    
    # Create a DataFrame with the calculated metrics, using 'model_name' as the column label.
    result_df = pd.DataFrame([MAE, MSE, RMSE, R2_Score], index=['MAE', 'MSE', 'RMSE', 'R2-Score'], columns=[model_name])
    
    # Return the DataFrame containing the metrics.
    return result_df

# Call the 'model_evaluation' function with the specified model, test data, and model name.
model_evaluation(regressor_lr, X_train, y_train, 'Support vector mchine')

,Support vector mchine
MAE,0.488312
MSE,0.877630
RMSE,0.936819
R2-Score,0.990601


In [22]:
def compare_plot(df_comp):
    #df_comp.reset_index(inplace=True)
    df_comp_first_100 = df_comp.iloc[50:150]  # Slice the DataFrame to include only the first 100 rows
    df_comp_first_100.plot(y=['CO2 Emissions(g/km)', 'Predicted'], kind='bar', figsize=(20, 7), width=0.8)
    plt.title('Predicted vs. Actual Target Values for Test Data (First 100 Rows)', fontsize=20)
    plt.ylabel('CO2 Emissions(g/km)', fontsize=15)
    plt.show()

compare_plot(y_pred)

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [24]:
y_test_pred = regressor_lr.predict(X_train)

# Create a DataFrame 'ytest_ored' with a single column named 'y_test_pred' to store the predictions.
ytrain_pred = pd.DataFrame({'y_test_pred': y_test_pred})

# Display the 'ytest_ored' DataFrame, showing the predicted values.
ytrain_pred

ValueError: Per-column arrays must each be 1-dimensional

In [57]:
y_predict = regressor_lr.predict(X_test)

# Create a DataFrame 'predict_df' with a single column named 'Predicted' to store the predictions.
predict_df = pd.DataFrame({'Predicted': y_predict})

# Display the 'predict_df' DataFrame, showing the predicted values.
predict_df

ValueError: Per-column arrays must each be 1-dimensional

In [25]:
y_predict

NameError: name 'y_predict' is not defined

In [61]:
pr = pd.DataFrame({'y_te':y_test, 'ypre':y_predict})

ValueError: Per-column arrays must each be 1-dimensional

In [62]:
denormalized_actual_val = scaler.inverse_transform(y_test_actual)

In [63]:
actual_one_dim_list = [element for row in denormalized_actual_val for element in row]
denormalized_actual_val_df=pd.DataFrame({'denormalized_actual_val': actual_one_dim_list})
denormalized_actual_val_df

,denormalized_actual_val
0,254.0
1,373.0
2,228.0
3,303.0
4,193.0
...,...
1477,218.0
1478,194.0
1479,288.0
1480,225.0


In [26]:
categorical_columns = ['Make', 'Transmission', 'Fuel Type']

user_input= {
    'Make': ['ACURA'],  # Include both Make and Model
    'Engine Size(L)': [2.0],  # Update the Engine Size
    'Cylinders': [4.0],
    'Transmission': ['AS5'],  # Update the Transmission
    'Fuel Type': ['Z'],
    'Fuel Consumption City (L/100 km)': [9.9],
    'Fuel Consumption Hwy (L/100 km)': [6.7],  # Update fuel consumption values
}

# Create a DataFrame from the user input
user_input_df = pd.DataFrame(user_input)

# One-hot encode the categorical variables to match X_train columns
user_input_encoded = pd.get_dummies(user_input_df, columns=categorical_columns)

# Initialize the user input DataFrame with all zeros and columns from X_train
user_input_for_prediction = pd.DataFrame(0, index=[0], columns=X_train.columns)

# Fill in the columns that match based on user input
for col in user_input_encoded.columns:
    if col in user_input_for_prediction.columns:
        user_input_for_prediction.loc[0, col] = user_input_encoded[col].iloc[0]

# Calculate 'Fuel Consumption Comb (L/100 km)' based on 'Fuel Consumption City (L/100 km)' and 'Fuel Consumption Hwy (L/100 km)'
user_input_for_prediction['Fuel Consumption Comb (L/100 km)'] = (0.55 * user_input_for_prediction['Fuel Consumption City (L/100 km)'] +
                                                              0.45 * user_input_for_prediction['Fuel Consumption Hwy (L/100 km)'])

# Calculate 'Fuel Consumption Comb (mpg)' based on 'Fuel Consumption Comb (L/100 km)'
user_input_for_prediction['Fuel Consumption Comb (mpg)'] = 282.481/ user_input_for_prediction['Fuel Consumption Comb (L/100 km)']

# Make predictions
predictions = regressor_lr.predict(user_input_for_prediction)

# The 'predictions' variable now contains the predicted values for the user input
print(predictions)

denormalized_userinput = scaler.inverse_transform(predictions.reshape(-1, 1))
print(denormalized_userinput)

[[16.46922476]]
[[196.22699639]]


In [27]:
import pickle as pk
# Specify the file path where you want to save the model
model_filename = 'C:\\Users\\HP\\Desktop\\Prediction App\\leaniermodel.pkl'

# Use pickle to save the model to a file
with open(model_filename, 'wb') as model_file:
    pk.dump(regressor_lr, model_file)

#FileLink(model_filename)